### Seleção de recursos para aprendizado de máquina

Os recursos de dados que você usa para treinar seus modelos de aprendizado de máquina têm um enorme
influência sobre o desempenho que você pode alcançar. Características irrelevantes ou parcialmente relevantes podem impactar negativamente desempenho do modelo. Neste capítulo, você descobrirá técnicas de seleção automática de recursos que você pode usar para preparar seus dados de aprendizado de máquina em Python com scikit-learn. 

1. Seleção Univariada.
2. Eliminação de características recursivas.
3. Análise de componentes principais.
4. Importância do recurso.

#### Seleção de Recursos

A seleção de recursos é um processo em que você seleciona automaticamente os recursos em seus dados que contribuem mais para a variável de previsão ou saída na qual você está interessado. Tendo recursos irrelevantes em seus dados podem diminuir a precisão de muitos modelos, especialmente lineares algoritmos como regressão linear e logística. Três benefícios de executar a seleção de recursos antes de modelar seus dados são:

* Reduz o over tting: dados menos redundantes significam menos oportunidades para tomar decisões
com base no ruído.
* Melhora a precisão: Dados menos enganosos significam melhora na precisão da modelagem.
* Reduz o tempo de treinamento: menos dados significa que os algoritmos treinam mais rápido.
* Você pode aprender mais sobre a seleção de recursos com o scikit-learn no artigo Seleção de recursos1.
* Cada receita de seleção de recurso usará o conjunto de dados de início do diabetes dos índios Pima.

#### Seleção Univariada

Os testes estatísticos podem ser usados para selecionar os recursos que têm o relacionamento mais forte com a variável de saída. A biblioteca scikit-learn fornece o SelectKBest class2 que pode ser usado com um conjunto de diferentes testes estatísticos para selecionar um número específico de recursos. O exemplo abaixo usa o teste estatístico qui-quadrado (chi2) para recursos não negativos para selecionar 4 dos melhores características do conjunto de dados de diabetes dos índios Pima.

In [1]:
# Feature Extraction with Univariate Statistical Tests (Chi-squared for classification)
from pandas import read_csv
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

import warnings
warnings.filterwarnings('ignore')

# load data
filename = 'datasets/diabetes.csv'
names = ['preg', 'plas', 'pres',
         'skin', 'test', 'mass', 
         'pedi', 'age', 'class']

dataframe = read_csv(filename, names=names)
array = dataframe.values

X = array[:,0:8]
Y = array[:,8]

# feature extraction
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, Y)

# summarize scores
set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(X)

# summarize selected features
print(features[0:5,:])

[ 111.52  1411.887   17.605   53.108 2175.565  127.669    5.393  181.304]
[[148.    0.   33.6  50. ]
 [ 85.    0.   26.6  31. ]
 [183.    0.   23.3  32. ]
 [ 89.   94.   28.1  21. ]
 [137.  168.   43.1  33. ]]


Você pode ver as pontuações de cada atributo e os 4 atributos escolhidos (aqueles com maior pontuações): plas, teste, massa e idade. Eu tenho os nomes para os atributos escolhidos manualmente mapeando o índice das 4 pontuações mais altas para o índice dos nomes dos atributos.

#### Eliminação de recursos recursivos

A Eliminação Recursiva de Recursos (ou RFE) funciona removendo recursivamente atributos e construindo um modelo sobre os atributos que permanecem. Ele usa a precisão do modelo para identificar quais atributos (e combinação de atributos) contribuem mais para prever o atributo de destino. Você pode aprender mais sobre o RFE class3 na documentação do scikit-learn. O exemplo abaixo usa RFE com o algoritmo de regressão logística para selecionar os 3 principais recursos. A escolha de algoritmo não importa muito, desde que seja hábil e consistente.

In [2]:
# Feature Extraction with RFE
from pandas import read_csv
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# load data
filename = 'datasets/diabetes.csv'
names = ['preg', 'plas', 'pres',
         'skin', 'test', 'mass', 
         'pedi', 'age', 'class']

dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# feature extraction
model = LogisticRegression()
rfe = RFE(model, 
          n_features_to_select=3, 
          step=1)
fit = rfe.fit(X, Y)

print(f"Num Features: {fit.n_features_}")
print(f"Selected Features: {fit.support_}")
print(f"Feature Ranking: {fit.ranking_}")

Num Features: 3
Selected Features: [ True False False False False  True  True False]
Feature Ranking: [1 2 4 5 6 1 1 3]


Você pode ver que o RFE escolheu os 3 principais recursos como preg, mass e pedi. Estes são marcados Verdadeiro na matriz de suporte e marcado com a opção 1 na matriz de classificação. Novamente, você pode mapeie manualmente os índices de recursos para os índices de nomes de atributos.

#### Análise do componente principal

A Análise de Componentes Principais (ou PCA) usa álgebra linear para transformar o conjunto de dados em um forma comprimida. Geralmente, isso é chamado de técnica de redução de dados. Uma propriedade do PCA é que você pode escolher o número de dimensões ou componentes principais no resultado transformado. Em No exemplo abaixo, usamos o PCA e selecionamos 3 componentes principais. Saiba mais sobre o PCA class no scikit-learn revisando o API.

In [3]:
# Feature Extraction with PCA
from pandas import read_csv
from sklearn.decomposition import PCA

# load data
filename = 'datasets/diabetes.csv'
names = ['preg', 'plas', 'pres',
         'skin', 'test', 'mass', 
         'pedi', 'age', 'class']

dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# feature extraction
pca = PCA(n_components=3)
fit = pca.fit(X)

# summarize components
print(f"Explained Variance: {fit.explained_variance_ratio_}")
print(fit.components_)

Explained Variance: [0.889 0.062 0.026]
[[-2.022e-03  9.781e-02  1.609e-02  6.076e-02  9.931e-01  1.401e-02
   5.372e-04 -3.565e-03]
 [-2.265e-02 -9.722e-01 -1.419e-01  5.786e-02  9.463e-02 -4.697e-02
  -8.168e-04 -1.402e-01]
 [-2.246e-02  1.434e-01 -9.225e-01 -3.070e-01  2.098e-02 -1.324e-01
  -6.400e-04 -1.255e-01]]


Você pode ver que o conjunto de dados transformado (3 componentes principais) apresenta pouca semelhança aos dados de origem.

#### Importância do recurso

Árvores de decisão ensacadas como Random Forest e Extra Trees podem ser usadas para estimar a importância de características. No exemplo abaixo construímos um classificador ExtraTreesClassifier para o Pima Conjunto de dados de início de diabetes na Índia. Você pode aprender mais sobre a classe ExtraTreesClassifier na API scikit-learn.

In [4]:
# Feature Importance with Extra Trees Classifier
from pandas import read_csv
from sklearn.ensemble import ExtraTreesClassifier

# load data
filename = 'datasets/diabetes.csv'
names = ['preg', 'plas', 'pres',
         'skin', 'test', 'mass', 
         'pedi', 'age', 'class']

dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

# feature extraction
model = ExtraTreesClassifier()
model.fit(X, Y)
print(model.feature_importances_)

[0.112 0.239 0.099 0.078 0.072 0.138 0.12  0.14 ]


Você pode ver que recebemos uma pontuação de importância para cada atributo onde quanto maior o pontuação, mais importante é o atributo. As pontuações sugerem a importância de plas, idade e massa.

In [5]:
%reload_ext watermark
%watermark -a "Caique Miranda" -gu "caiquemiranda" -iv

Author: Caique Miranda

Github username: caiquemiranda



### End.